# Complete Bend Process

This notebook contains the entire process for creating intersection-scale ridge measurements (ridge amplitude, width, and spacing) from the required input data (dem, manual ridge lines, packets, bend area, centerline). This process is the same process that has been split up across the other 3 notebooks, but without the context and explanation in markdown cells.


In [ ]:
from pathlib import Path

import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt

from parameters import RASTER_WINDOW_SIZE, SMOOTHING_WINDOW_SIZE, SMALL_FEATS_SIZE, ELONGATION_THRESHOLD, VERTEX_SPACING, SHOOT_DISTANCE, SEARCH_DISTANCE, DEV_FROM_90
from scrollstats import CalcProfileCurvature, CalcResidualTopography, BinaryClassifier, RasterAgreementAssessor, RasterClipper, RasterDenoiser, LineSmoother, create_transects, calculate_ridge_metrics

In [ ]:
# Bend ID
bend_id = "LBR_025"

# Output Directory
output_dir = Path("example_data/output")

# Raster Paths
dem_path = Path(f"example_data/input/{bend_id}_dem.tif")
profc_path = Path(f"example_data/output/{bend_id}_dem_profc{RASTER_WINDOW_SIZE}px.tif")

# Vector Paths
bend_path = Path(f"example_data/input/{bend_id}_bend.geojson")
packet_path = Path(f"example_data/input/{bend_id}_packets.geojson")
centerline_path = Path(f"example_data/input/{bend_id}_cl.geojson")
manual_ridge_path = Path(f"example_data/input/{bend_id}_ridges_manual.geojson")


# Print constants from parameters.py
print(f"{RASTER_WINDOW_SIZE=}")
print(f"{SMOOTHING_WINDOW_SIZE=}")
print(f"{SMALL_FEATS_SIZE=}")
print(f"{ELONGATION_THRESHOLD=}")
print(f"{VERTEX_SPACING=}")
print(f"{SHOOT_DISTANCE=}")
print(f"{SEARCH_DISTANCE=}")
print(f"{DEV_FROM_90=}")


# Delineate Rasters

In [ ]:
profc = CalcProfileCurvature(dem_path, RASTER_WINDOW_SIZE, output_dir)
profc_path = profc.execute()

print(f"Profile Curvature raster saved to {profc_path}")

In [ ]:
rt = CalcResidualTopography(dem_path, RASTER_WINDOW_SIZE, output_dir)
rt_path = rt.execute()

In [ ]:
raster_paths = [profc_path, rt_path]
binclass_paths = []
for path in raster_paths:
    bc = BinaryClassifier(path, 0, output_dir)
    binclass_path = bc.execute()
    binclass_paths.append(binclass_path)

In [ ]:
# Set parameters
profc_binclass_path, rt_binclass_path = binclass_paths

ra = RasterAgreementAssessor(profc_binclass_path, rt_binclass_path, bend_id, output_dir)
agreement_path = ra.execute()

In [ ]:
bend = gpd.GeoDataFrame.from_file(bend_path)

to_clip_paths = [dem_path, agreement_path]

clip_paths = []
for path in to_clip_paths:

    geom = bend["geometry"][0] # RasterClipper requires a shapely polygon, not geodataframe

    rc = RasterClipper(path, geom, output_dir)
    clip_path = rc.execute()
    
    clip_paths.append(clip_path)

dem_clip_path, agreement_clip_path = clip_paths

In [ ]:
# Denoise Agreement Raster
dn = RasterDenoiser(agreement_clip_path, SMALL_FEATS_SIZE, ELONGATION_THRESHOLD, output_dir)
denoise_path = dn.execute()

In [ ]:
# Plot denoising results
agr = rasterio.open(agreement_clip_path).read(1)
denoise = rasterio.open(denoise_path).read(1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

mapper = ax1.imshow(agr)
ax1.set_axis_off()
ax1.set_title("Agreement")

mapper = ax2.imshow(denoise)
ax2.set_axis_off()
ax2.set_title("Denoised");

# Create Vector Datasets

In [ ]:
manual_ridges = gpd.read_file(manual_ridge_path)
cl = gpd.read_file(centerline_path)
packets = gpd.read_file(packet_path).set_index("packet_id")

In [ ]:
# Smooth and densify the lines
ls = LineSmoother(manual_ridges, VERTEX_SPACING, SMOOTHING_WINDOW_SIZE)
smooth_ridges = ls.execute()

# Save smooth ridges to disk
output_dir = Path("example_data/output")
smooth_ridge_name = manual_ridge_path.with_stem(manual_ridge_path.stem + "_smoothed").name
smooth_ridge_path = output_dir / smooth_ridge_name

smooth_ridges.to_file(smooth_ridge_path, driver="GeoJSON", index=False)

# Plot manual and smoothed lines for comparison
m = manual_ridges.explore(color="black", style_kwds={"weight":5})
smooth_ridges.explore(color="red", m=m)

In [ ]:
# define the distance between transects 
step = 100

# With a vertex spacing of ~1m, take every `step`th vertex along the centerline
starts = np.asarray(cl.geometry[0].xy).T[::step]

# Transect Parameters
shoot_distance = SHOOT_DISTANCE       # Distance that the N1 coordinate will shoot out from point P1; measured in linear unit of dataset
search_distance = SEARCH_DISTANCE     # Buffer radius used to search for an N2 coordinate on R2; measured in linear unit of dataset
dev_from_90 = DEV_FROM_90             # Max angular deviation from 90° allowed when searching for an N2 coordinate on R2; measured in degrees

transects = create_transects(cl, smooth_ridges, step, shoot_distance, search_distance, dev_from_90)

# Save transects to disk
transect_path = output_dir / f"{bend_id}_transects.geojson"
transects.to_file(transect_path, driver="GeoJSON", index=True)

# Calculate Ridge Metrics

In [ ]:
# Vector Data
ridges = gpd.read_file(smooth_ridge_path)
transects = gpd.read_file(transect_path)
packets = gpd.read_file(packet_path)
cl = gpd.read_file(centerline_path)

# Raster Data
bin_raster = rasterio.open(agreement_clip_path)
dem = rasterio.open(dem_clip_path)

In [ ]:
rich_transects, itx = calculate_ridge_metrics(transects, ridges, bin_raster, dem)
itx = itx.loc["LBR_025"]

# Add packets
itx_w_packets = itx.sjoin(packets.drop("bend_id", axis=1))
itx_w_packets = itx_w_packets.reset_index().set_index(["transect_id", "ridge_id", "packet_id"])
ridge_metrics_w_packets = itx_w_packets[["ridge_amp", "ridge_width", "pre_mig_dist", "geometry"]]
ridge_metrics_w_packets.columns = ridge_metrics_w_packets.columns.rename("metrics")

# Save to disk
itx_path = output_dir / f"{bend_id}_intersections.geojson"
ridge_metrics_w_packets.to_file(itx_path, driver="GeoJSON", index=True)

In [ ]:
# Plot itx
fig, ax = plt.subplots(1,1, figsize=(10,6))
ridges.plot(ax=ax, color="k", ls="--", lw=0.5, zorder=0)
transects.plot(ax=ax, color="k", lw=1, zorder=1)
cl.plot(ax=ax, color="tab:blue", lw=5, zorder=2)

itx_w_packets.plot(column="ridge_amp", ax=ax, zorder=2, legend=True, 
                   legend_kwds={"label": "Ridge Amplitude [m]"})

ax.set_title("Ridge amplitude at each intersection")